In [3]:
import numpy as np
import dynet as dy


In [2]:
def affine(m, h, U):
    _, c, d = U.shape
    return h @ (m @ U.reshape(d, -1)).reshape(d, c)

def biaffine(m, h, U, W, b):        
    return affine(m, h, U) + cat([m,h]) @ W + b

In [25]:
def affine(m, h, U):
    (_, c, d), _ = U.dim()
    a = dy.reshape(U, (d, d*c))
    r = dy.reshape(dy.transpose(m) * a, (d, c))
    return dy.transpose(h) * r

def biaffine(m, h, U, W, b):        
    return affine(m, h, U) + (dy.reshape(dy.concatenate([m,h]), (1,200)) *  W + b)

In [34]:
c, d = 1, 100
m = dy.inputTensor(np.random.rand(d))
h = dy.inputTensor(np.random.rand(d))
U = dy.inputTensor(np.random.rand(d, c, d))
W = dy.inputTensor(np.random.rand(d*2, c))
b = dy.inputTensor(np.random.rand(1,c))

In [35]:
affine(m, h, U).dim()

((1, 1), 1)

In [36]:
biaffine(m, h, U, W, b).dim()

((1, 1), 1)

affine: $$a(x_1, x_2) = x_1^\intercal U x_2$$

biaffine: 
$$b(x_1, x_2) = x_1^\intercal U x_2 + W(x_1 \oplus x_2) + b$$
$$=$$
$$b(x_1, x_2) = a(x_1, x_2) + W(x_1 \oplus x_2) + b$$


*Numpy*
```python
def affine(m, h, U):
    _, c, d = U.shape
    return h @ (m @ U.reshape(d, -1)).reshape(d, c)
    
def biaffine(m, h, U, W, b):        
    return affine(m, h, U) + cat([m,h]) @ W + b
```
(Note that this take into account the higher-dimensional setup where U is `(d,l,d)` with `l > 1`)

In [74]:
# time to simplify
params = dy.ParameterCollection()
c, d = 50, 100
m = dy.inputTensor(np.random.rand(d))
h = dy.inputTensor(np.random.rand(d))
U = params.add_lookup_parameters((d, d, c))
W = params.add_lookup_parameters((c, d*2))
b =  params.add_lookup_parameters((1,c))

In [71]:
def affine(m, h, U):
    (_, c, d), _ = U.dim()
    a = dy.reshape(U, (d, d*c))
    r = dy.reshape(dy.transpose(m) * a, (d, c))
    return dy.transpose(h) * r

def biaffine(m, h, U, W, b):        
    return affine(m, h, U) + (dy.reshape(dy.concatenate([m,h]), (1,200)) *  W + b)

In [72]:
biaffine(m, h, U, W, b).dim()

((50, 50), 1)

# get parent

In [11]:
from collections import defaultdict
pm = defaultdict(list)
n = 10
golds = np.random.randint(0, n, 10)
f = lambda a,b: a+b
for m in range(n):
    for h in range(n):
        f(i,j) + golds[h]

In [12]:
pm

defaultdict(list,
            {8: [0], 0: [1, 2, 8], 7: [3], 3: [4, 7], 9: [5], 1: [6], 6: [9]})

In [13]:
print(np.arange(n))
print(golds)

[0 1 2 3 4 5 6 7 8 9]
[8 0 0 7 3 9 1 3 0 6]


# attention
1. I'd like to feed its parent in